# Posture Scoring

In [2]:
!pip install "opencv-python-headless<4.3"

In [3]:
import numpy as np

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def calculate_angle(point1, point2, point3):
    """Calculate angle between three points (in degrees).
       When considering the points, only use the (x, y) coordinates
    """
    a = np.array(point1)  # Convert point1 to a NumPy array
    b = np.array(point2)  # Convert point2 to a NumPy array
    c = np.array(point3)  # Convert point3 to a NumPy array

    # Calculate vectors from point B to point A and from point B to point C
    ba = a - b
    bc = c - b

    # Calculate the cosine of the angle between the vectors
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    
    # Ensure cosine_angle is within -1 to 1 range to avoid NaN results
    cosine_angle = np.clip(cosine_angle, -1, 1)

    # Calculate the angle in radians and then convert to degrees
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)
    return np.degrees(angle)

def normalize_score(angle, ideal_angle):
    """Normalize score between 0 and 1, where 0 is the ideal_angle."""
    deviation = abs(angle - ideal_angle)
    normalized_score = deviation / ideal_angle
    return min(normalized_score, 1)  # Ensure score does not exceed 1

def heuristic_posture_scores(keypoints, FRAME_HEIGHT=1080):
    """Score the postural, spinal, and shoulder alignment.
    Assuming keypoints: 0-Nose, 5-Left Shoulder, 6-Right Shoulder, 11-Left Hip, 12-Right Hip"""

    # Calculate angles
    back_angle = calculate_angle(keypoints[10:12], keypoints[22:24], keypoints[24:26])  # Shoulder to hips
    neck_head_angle = calculate_angle(keypoints[0:2], keypoints[10:12], keypoints[12:14])  # Nose to shoulders

    # Shoulder levelness: Ideal is a straight horizontal line, so difference in y-coordinates
    shoulder_levelness = abs(keypoints[10] - keypoints[11])

    # Assuming the maximum possible y-coordinate difference as the height of the frame (e.g., 1080 pixels),
    # you may adjust this based on your actual frame height
    shoulder_score = min(shoulder_levelness / FRAME_HEIGHT, 1)

    # Normalize scores (0 is best, 1 is worst)
    back_score = normalize_score(back_angle, 180)
    neck_head_score = normalize_score(neck_head_angle, 180)

    return {'back_score': back_score, 'neck_head_score': neck_head_score, 'shoulder_score': shoulder_score}

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# Train regression model on real data points from yoga_train_data.csv

df = pd.read_csv('/Users/jennycai/Desktop/TreeHacks2024/ml/sample_data/yoga_train_data.csv') # TODO: CHANGE TO RELATIVE PATH
cols_of_interest = [
    'nose_x', 'nose_y', 'nose_score',
    'left_eye_x', 'left_eye_y', 'left_eye_score',
    'right_eye_x', 'right_eye_y', 'right_eye_score',
    'left_ear_x', 'left_ear_y', 'left_ear_score',
    'right_ear_x', 'right_ear_y', 'right_ear_score',
    'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_score',
    'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_score',
    'left_elbow_x', 'left_elbow_y', 'left_elbow_score',
    'right_elbow_x', 'right_elbow_y', 'right_elbow_score',
    'left_wrist_x', 'left_wrist_y', 'left_wrist_score',
    'right_wrist_x', 'right_wrist_y', 'right_wrist_score',
    'left_hip_x', 'left_hip_y', 'left_hip_score',
    'right_hip_x', 'right_hip_y', 'right_hip_score',
    'left_knee_x', 'left_knee_y', 'left_knee_score',
    'right_knee_x', 'right_knee_y', 'right_knee_score',
    'left_ankle_x', 'left_ankle_y', 'left_ankle_score',
    'right_ankle_x', 'right_ankle_y', 'right_ankle_score'
]
cols_of_interest = [col for col in cols_of_interest if not col.endswith('_score')]

df = df.filter(cols_of_interest)
X = df.to_numpy() # (968, 34)
num_samples = X.shape[0]

y_back, y_shoulder, y_neck_head = [], [], []
# Simulate scores for back, shoulder, and neck/head alignment (ranging from 0 to 1)
for keypoints in X:
  curr_y_back, curr_y_neck_head, curr_y_shoulder = heuristic_posture_scores(keypoints).values()
  y_back.append(curr_y_back)
  y_shoulder.append(curr_y_shoulder)
  y_neck_head.append(curr_y_neck_head)

y_back, y_shoulder, y_neck_head = np.array(y_back).reshape((num_samples, 1)), np.array(y_shoulder).reshape((num_samples, 1)), np.array(y_neck_head).reshape((num_samples, 1))

# Split data into training and testing sets
X_train, X_test, y_train_back, y_test_back = train_test_split(X, y_back, test_size=0.2, random_state=42)
_, _, y_train_shoulder, y_test_shoulder = train_test_split(X, y_shoulder, test_size=0.2, random_state=42)
_, _, y_train_neck_head, y_test_neck_head = train_test_split(X, y_neck_head, test_size=0.2, random_state=42)

In [5]:
# Train separate models for each score
model_back = LinearRegression().fit(X_train, y_train_back)
model_shoulder = LinearRegression().fit(X_train, y_train_shoulder)
model_neck_head = LinearRegression().fit(X_train, y_train_neck_head)

# Movenet

In [6]:
!pip install opencv-python
!pip install mediapipe
!pip install tensorflow
!pip install tensorflow-hub

In [7]:
#Computer vision/graphics library
import cv2

# Gif writer
import imageio

# Display libraries
import matplotlib.pyplot as plt
from IPython.display import HTML, display

# Calculations and Deep Learning library
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

2024-02-17 14:14:41.006691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Set up colors + size

In [8]:
blue = (173, 216, 230)
green = (144, 238, 144)

EDGE_COLORS = {
    (0, 1): green,
    (0, 2): blue,
    (1, 3): green,
    (2, 4): blue,
    (0, 5): green,
    (0, 6): blue,
    (5, 7): green,
    (7, 9): blue,
    (6, 8): green,
    (8, 10): blue,
    (5, 6): green,
    (5, 11): blue,
    (6, 12): green,
    (11, 12): blue,
    (11, 13): green,
    (13, 15): blue,
    (12, 14): green,
    (14, 16): blue
}

In [9]:
#initial_width, initial_height = (461,250)
WIDTH = HEIGHT = 256

### Load Model

In [10]:
model = hub.load("https://tfhub.dev/google/movenet/multipose/lightning/1")
movenet = model.signatures["serving_default"]

### Define the loop
Steps : loop through the results ---> Draw the keypoints ----> Draw the edges


In [11]:
def loop(frame, keypoints, threshold=0.11):
    """
    Main loop : Draws the keypoints and edges for each instance
    """

    # Loop through the results
    for instance in keypoints:
        # Draw the keypoints and get the denormalized coordinates
        denormalized_coordinates = draw_keypoints(frame, instance, threshold)
        # Draw the edges
        draw_edges(denormalized_coordinates, frame, EDGE_COLORS, threshold)

### Draw points

In [12]:
def draw_keypoints(frame, keypoints, threshold=0.11):
    """Draws the keypoints on a image frame"""

    # Denormalize the coordinates : multiply the normalized coordinates by the input_size(width,height)
    denormalized_coordinates = np.squeeze(np.multiply(keypoints, [WIDTH,HEIGHT,1]))
    #Iterate through the points
    for keypoint in denormalized_coordinates:
        # Unpack the keypoint values : y, x, confidence score
        keypoint_y, keypoint_x, keypoint_confidence = keypoint
        if keypoint_confidence > threshold:
            """"
            Draw the circle
            Note : A thickness of -1 px will fill the circle shape by the specified color.
            """
            cv2.circle(
                img=frame,
                center=(int(keypoint_x), int(keypoint_y)),
                radius=4,
                color=(255,0,0),
                thickness=-1
            )
    return denormalized_coordinates

### Draw edges

In [13]:
def draw_edges(denormalized_coordinates, frame, edges_colors, threshold=0.11):
    """
    Draws the edges on a image frame
    """

    # Iterate through the edges
    for edge, color in edges_colors.items():
        # Get the dict value associated to the actual edge
        p1, p2 = edge
        # Get the points
        y1, x1, confidence_1 = denormalized_coordinates[p1]
        y2, x2, confidence_2 = denormalized_coordinates[p2]
        # Draw the line from point 1 to point 2, the confidence > threshold
        if (confidence_1 > threshold) & (confidence_2 > threshold):
            cv2.line(
                img=frame,
                pt1=(int(x1), int(y1)),
                pt2=(int(x2), int(y2)),
                color=color,
                thickness=2,
                lineType=cv2.LINE_AA # Gives anti-aliased (smoothed) line which looks great for curves
            )


### Process real-time frames

In [14]:
def initialize_webcam():
    """
    Initializes webcam for capturing and returns necessary objects for processing.
    """
    # Initialize webcam capture; 0 usually refers to the default webcam.
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return None, None, None, None

    # Assuming frame count is not fixed for a continuous stream
    frame_count = None
    
    # Initialize an empty list to hold processed frames (if needed)
    output_frames = []

    # Get initial shape (width, height) from the first frame to setup processing parameters
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        cap.release()
        return None, None, None, None
    
    initial_shape = [frame.shape[1], frame.shape[0]]  # Width, Height

    # Return to the beginning of the stream
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    return cap, frame_count, output_frames, initial_shape


### Run inference

Connects to the webcam, and displays your live skeleton estimate alongside a real-time posture scoring.

In [ ]:
import cv2
import imageio
from IPython.display import display, Image  # Assuming this is run in a Jupyter Notebook or similar environment
import time

def process_webcam_input(movenet, WIDTH=192, HEIGHT=192, MAX_FRAMES=1000):
    """
    Continuously captures frames from the webcam, displays the estimated skeleton in real-time.
    """
    cap, frame_count, output_frames, initial_shape = initialize_webcam()
    if cap is None:
        print("Webcam not accessible.")
        return

    output_frames = []

    print("Starting real-time inference...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Process frame
        image = cv2.resize(frame, (WIDTH, HEIGHT))
        input_image = tf.cast(tf.image.resize_with_pad(image, WIDTH, HEIGHT), dtype=tf.int32)
        input_image = tf.expand_dims(input_image, axis=0)

        # Run inference
        results = movenet(input_image)
        """
        Output shape :  [1, 6, 56] ---> (batch size), (instances), (xy keypoints coordinates and score from [0:50]
        and [ymin, xmin, ymax, xmax, score] for the remaining elements)
        First, let's resize it to a more convenient shape, following this logic :
        - First channel ---> each instance
        - Second channel ---> 17 keypoints for each instance
        - The 51st values of the last channel ----> the confidence score.
        Thus, the Tensor is reshaped without losing important information.
        """
        keypoints = results["output_0"].numpy()[:,:,:51].reshape((6,17,3))

        loop(image, keypoints, threshold=0.11)

        # Get the output frame : reshape to the original size
        frame_rgb = cv2.cvtColor(
            cv2.resize(
                image,(initial_shape[0], initial_shape[1]),
                interpolation=cv2.INTER_LANCZOS4
            ),
            cv2.COLOR_BGR2RGB # OpenCV processes BGR images instead of RGB
        )
        
        # REAL-TIME OUTPUTS
        
        # skeleton frames overlayed on webcame output
        cv2.imshow('Skeleton', frame_rgb)
        
        # posture scores printed to console
        keypoints_yx = keypoints[0, :, :2] # Select the first batch of keypoints and then take only the (y, x) coordinates, discarding the confidence scores
        
        # Flatten the array to have a shape of (1, 34)
        keypoints_processed = keypoints_yx.flatten().reshape(1, -1)
        
        score_back = model_back.predict(keypoints_processed)
        score_shoulder = model_shoulder.predict(keypoints_processed)
        score_neck_head = model_neck_head.predict(keypoints_processed)

        print(f"Predicted Back Alignment Score: {score_back[0][0]:.2f}")
        print(f"Predicted Shoulder Alignment Score: {score_shoulder[0][0]:.2f}")
        print(f"Predicted Neck/Head Alignment Score: {score_neck_head[0][0]:.2f}")
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage, assuming 'movenet' is your loaded TensorFlow model
process_webcam_input(movenet)